In [2]:
"""
currently checking the working of data loader and also sved in dataloader.py file the update the work
"""

import os
import tensorflow as tf
import cv2
import numpy as np

In [3]:
image_path_dir = "/home/roboticist/Documents/Swaayatt/swaayatt_optical_flow/dataset_undeepVO/data_scene_flow(1)/training"
calib_dir = "/home/roboticist/Documents/Swaayatt/swaayatt_optical_flow/dataset_undeepVO/data_scene_flow(1)/data_scene_flow_calib/training/calib_cam_to_cam/"

In [5]:
left = sorted(os.listdir(image_path_dir+"/image_2"))
right = sorted(os.listdir(image_path_dir+"/image_3"))
calib_file_list = sorted(os.listdir(calib_dir))
#len(a)
#print(left)

In [23]:
def data_loader(left_image_dir,right_image_dir,image_width,image_height,intrinsic_dir,batch=2,number_of_data=None):
    training_data = []
    left = sorted(os.listdir(left_image_dir))
    right = sorted(os.listdir(right_image_dir))
    calib_file_list = sorted(os.listdir(intrinsic_dir))
    ##load the intrinsic parameter first
    ##intrinsic parameter data loader
    calib_file_list = sorted(os.listdir(intrinsic_dir))
    file = open(calib_dir+calib_file_list[0])
    #intrinsic_row = []
    intrinsic_list = []
    for line in file:
        data = line.split()
        if 'K_03:'==data[0]:
            for d in data[1:]:
                intrinsic_list.append(float(d))
                
    intr = np.array(intrinsic_list,dtype = np.float32).reshape(3,3)
    intrinsic = tf.convert_to_tensor(intr,tf.float32)
    
    if number_of_data == None:
        N = len(left)-2
    else:
        N=number_of_data
    idx=0
    while idx<N:
        #if idx>=N:
            #break
        for j in  range(batch):
            left_imgk = tf.image.decode_png(tf.io.read_file(left_image_dir+left[idx+j]))
            left_imgk = tf.image.convert_image_dtype(left_imgk, tf.float32)
            l_k = tf.image.resize(left_imgk,  [image_height, image_width])
            left_imgk1 = tf.image.decode_png(tf.io.read_file(left_image_dir+left[idx+1+j]))
            left_imgk1 = tf.image.convert_image_dtype(left_imgk1, tf.float32)
            l_k1 = tf.image.resize(left_imgk1,  [image_height, image_width])
            
            #if idx==1:
                #cv2.imshow('image',left_imgk.numpy())            
            #left_imgk1 = cv2.imread(left_image_dir+left[idx+1+j])
            #left_imgk1 = cv2.resize(left_imgk1,(image_width,image_height))
            right_imgk = tf.image.decode_png(tf.io.read_file(right_image_dir+left[idx+j]))
            right_imgk = tf.image.convert_image_dtype(right_imgk, tf.float32)
            r_k = tf.image.resize(right_imgk,  [image_height, image_width])
            right_imgk1 = tf.image.decode_png(tf.io.read_file(right_image_dir+left[idx+1+j]))
            right_imgk1 = tf.image.convert_image_dtype(right_imgk1, tf.float32)
            r_k1 = tf.image.resize(right_imgk1,  [image_height, image_width])
            #left_imgk = cv2.imread(left_image_dir+left[idx+j])
            #left_imgk = cv2.resize(left_imgk,(image_width,image_height))
            #left_imgk1 = cv2.imread(left_image_dir+left[idx+1+j])
            #left_imgk1 = cv2.resize(left_imgk1,(image_width,image_height))
            #right_imgk = cv2.imread(right_image_dir+right[idx+j])
            #right_imgk = cv2.resize(right_imgk,(image_width,image_height))
            #right_imgk1 = cv2.imread(right_image_dir+right[idx+1+j])
            #right_imgk1 = cv2.resize(right_imgk1,(image_width,image_height))
            #l_k = tf.constant(left_imgk,tf.float32)
            #l_k1 = tf.constant(left_imgk1,tf.float32)
            #r_k = tf.constant(right_imgk,tf.float32)
            #r_k1 = tf.constant(right_imgk1,tf.float32)
            #print(left_image_dir+left[idx+j])
            #if idx==1:
                #cv2.imshow('image',l_k.numpy())
                #cv2.waitKey(0)
                #cv2.destroyAllWindows()
            train_left_image = tf.concat([l_k,l_k1],axis=2)
            train_right_image = tf.concat([r_k,r_k1],axis=2)
            #if idx==1:
            #    left_image = tf.slice(train_left_image,[0,0,0],[-1,-1,3])
            #    x = left_image.numpy().reshape(img_height,img_width,3).astype(np.float32)
                #cv2.imshow('image',x)
                #cv2.waitKey(0)
                #cv2.destroyAllWindows()
                #print(x.shape)
                #print(left.shape)
            #print(train_left_image.shape)
            #print(train_left_image.shape)
            if j==0:
                train_batch_left = tf.expand_dims(train_left_image,axis=0)
                train_batch_right = tf.expand_dims(train_right_image,axis=0)
            else:
                train_left_image = tf.expand_dims(train_left_image,axis=0)
                train_right_image = tf.expand_dims(train_right_image,axis=0)
                train_batch_left = tf.concat([train_batch_left,train_left_image],axis=0)
                train_batch_right = tf.concat([train_batch_right,train_right_image],axis=0)
            idx+=1
            if idx>=N:
                break
        if idx<N:
            training_data.append([train_batch_left,train_batch_right])
    return training_data,intrinsic
    

In [25]:
path_dir = "/home/roboticist/Documents/Swaayatt/swaayatt_optical_flow/dataset_undeepVO/data_scene_flow(1)/training"
left_img_dir = path_dir+"/image_2/"
right_img_dir = path_dir+"/image_3/"
img_width = 416
img_height = 128

train_ds,intrinsic = data_loader(left_img_dir,right_img_dir,img_width,img_height,calib_dir,batch=1,number_of_data=22)
#image = cv2.imread("/home/roboticist/Documents/Swaayatt/swaayatt_optical_flow/dataset_undeepVO/data_scene_flow(1)/training/image_2/000000_10.png")
#cv2.imshow("image",image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()


In [62]:
for i,(left,right) in enumerate(train_ds):
    if i==1:
        left_image = tf.slice(left,[0,0,0,0],[-1,-1,-1,3])
        x = left_image.numpy().reshape(img_height,img_width,3).astype(np.float32)
        #print(x.shape)
        #print(left.shape)
        cv2.imshow("image",x)
cv2.waitKey(0)
cv2.destroyAllWindows()
    

In [26]:
print(intrinsic)

tf.Tensor(
[[903.7596   0.     695.7519]
 [  0.     901.9653 224.2509]
 [  0.       0.       1.    ]], shape=(3, 3), dtype=float32)


In [17]:
##intrinsic parameter data loader
calib_file_list = sorted(os.listdir(calib_dir))
file = open(calib_dir+calib_file_list[0])

In [18]:
#intrinsic_row = []
intrinsic = []
for line in file:
    data = line.split()
    if 'K_03:'==data[0]:
        for d in data[1:]:
            intrinsic.append(float(d))

In [20]:
a = np.array(intrinsic,dtype = np.float32).reshape(3,3)

In [22]:
tensor = tf.convert_to_tensor(a,dtype=tf.float32)
print(tensor)

tf.Tensor(
[[903.7596   0.     695.7519]
 [  0.     901.9653 224.2509]
 [  0.       0.       1.    ]], shape=(3, 3), dtype=float32)
